In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
#Load MoveNet model
model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')
movenet = model.signatures['serving_default']

In [3]:
#  Keypoint extraction functions
def process_image(image):
    """Process image for MoveNet input"""
    image = tf.image.resize_with_pad(image, 192, 192)
    return tf.cast(image, dtype=tf.int32)

def extract_keypoints(image):
    """Extract keypoints from image using MoveNet"""
    image = tf.convert_to_tensor(image)
    image = process_image(image)
    image = tf.expand_dims(image, axis=0)
    outputs = movenet(image)
    return outputs['output_0'].numpy()[0][0]

In [4]:
#  Prepare training data (assuming images organized in class folders)
def prepare_dataset(data_dir):
    X = []
    y = []
    class_names = sorted(os.listdir(data_dir))
    
    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Extract keypoints and flatten
            keypoints = extract_keypoints(img)
            X.append(keypoints.flatten())
            y.append(class_idx)
    
    return np.array(X), np.array(y), class_names


In [5]:
# Replace with your dataset path
X, y, class_names = prepare_dataset('C:/Users/5A_Traders/Downloads/FYP_ON_DEV/FYP_IntelliTrain/PoseEstimation/Dataset')

In [6]:
# Step 6: Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
# Build classifier model
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(51,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

classifier.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

C:\Users\5A_Traders\miniconda3\envs\GTM\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Train classifier
history = classifier.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.7706 - val_accuracy: 0.0000e+00 - val_loss: 0.8411
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.4286 - loss: 0.7548 - val_accuracy: 0.0000e+00 - val_loss: 0.9081
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.7143 - loss: 0.6053 - val_accuracy: 0.0000e+00 - val_loss: 0.9547
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.7143 - loss: 0.5933 - val_accuracy: 0.0000e+00 - val_loss: 1.0102
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.5714 - loss: 0.6866 - val_accuracy: 0.0000e+00 - val_loss: 1.0502
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.6429 - loss: 0.6972 - val_accuracy: 0.0000e+00 - val_loss: 1.0719


In [9]:
# Step 9: Live prediction with webcam
def live_prediction(class_names):
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process frame
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        original_height, original_width = frame.shape[:2]
        
        # Extract keypoints
        keypoints = extract_keypoints(rgb_frame)
        features = keypoints.flatten().reshape(1, -1)
        
        # Predict class
        proba = classifier.predict(features, verbose=0)
        class_idx = np.argmax(proba)
        confidence = np.max(proba)
        
        # Calculate keypoint transformations
        scale = min(192 / original_height, 192 / original_width)
        new_height = int(original_height * scale)
        new_width = int(original_width * scale)
        pad_top = (192 - new_height) // 2
        pad_left = (192 - new_width) // 2
        
        # Draw keypoints
        for y_norm, x_norm, conf in keypoints:
            if conf > 0.3:
                x_pad = x_norm * 192
                y_pad = y_norm * 192
                x = int((x_pad - pad_left) / scale)
                y = int((y_pad - pad_top) / scale)
                
                # Draw circle if within frame bounds
                if 0 <= x < original_width and 0 <= y < original_height:
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        
        # Display prediction
        cv2.putText(frame, f"{class_names[class_idx]} {confidence:.2f}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow('Pose Estimation', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Start live prediction
live_prediction(class_names)